In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import pickle
import time
from torch.utils.data import DataLoader
import optuna
from transformers import BertTokenizer, VisualBertForPreTraining
import numpy as np

In [2]:
data_dir = r'E:\datasets\MADE\3_graduation\parthplc\archive\data\\'

train_path = data_dir + 'train.jsonl'
dev_path = data_dir + 'dev.jsonl'


train_data = pd.read_json(train_path, lines=True)
test_data = pd.read_json(dev_path, lines=True)

test_data.head(3)

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...


In [3]:
with open('d:\\visual_embeddings_val.pkl', 'rb') as f:
    visual_embeddings_val = pickle.load(f)
    
val_dict = {}
for x in test_data.values:
    if x[1] in visual_embeddings_val:
        val_dict[x[1]] = {'label':x[2], 'text':x[3], 'id':x[1]}


In [4]:
with open('d:\\visual_embeddings_train.pkl', 'rb') as f:
    visual_embeddings_train = pickle.load(f)
    
train_dict = {}
for x in train_data.values:
    if x[1] in visual_embeddings_train:
        train_dict[x[1]] = {'label':x[2], 'text':x[3], 'id':x[1]}


In [5]:
class FeaturesDataset(torch.utils.data.Dataset):
    def __init__(self, visual_embeddings, labels):
        self.visual_embeddings = visual_embeddings
        self.labels = labels
        
        self.idx2id = [{'id':k, 'label':labels[k]['label'], 'text':labels[k]['text']}
                       for i, k in enumerate(labels)]
    
    
    def __getitem__(self, index: int):
        id = self.idx2id[index]['id']
        return id, self.visual_embeddings[id][0], self.labels[id]['text'][:77], self.labels[id]['label']

    
    def __len__(self):
        return len(self.idx2id)

In [6]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, scheduler, device):
    t1 = time.time()
    best_model_name = None
    loss_history = []
    train_history = []
    val_history = []
    top_val_accuracy = 0.64 
    for epoch in range(num_epochs):
        model.train()
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (id, visual_embeds, text, y) in enumerate(train_loader):
            y = y.to(device)
            visual_embeds = visual_embeds.to(device)    
            tokens = tokenizer(list(text), padding='max_length', max_length=77)

            input_ids = torch.tensor(tokens["input_ids"], device=device)
            attention_mask = torch.tensor(tokens["attention_mask"], device=device)
            token_type_ids = torch.tensor(tokens["token_type_ids"], device=device)

            visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
            visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)


            outputs = model(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              token_type_ids=token_type_ids, 
                              visual_embeds=visual_embeds, 
                              visual_attention_mask=visual_attention_mask, 
                              visual_token_type_ids=visual_token_type_ids
                          )
    
            prediction = outputs.prediction_logits.sum(axis=1)
            
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader, device)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        if scheduler != None:
            scheduler.step()

        print("Epoch: %i; %.2f sec; lr: %f; Average loss: %.2f, Train accuracy: %.4f, Val accuracy: %.4f" % 
              (epoch, round(time.time() - t1, 2), get_lr(optimizer), ave_loss, train_accuracy, val_accuracy))

  
        if val_accuracy > top_val_accuracy:
            top_val_accuracy = val_accuracy
            model_name = f'classifier_{epoch}_{round(val_accuracy, 3)}.ckpt'
            best_model_name = model_name
            torch.save(model, open(model_name, 'wb'))
            print("saved", model_name)

        if len(val_history) > 4:
            print(f'{(val_history[-1] - val_history[-2]) < 0.001} {(val_history[-2] - val_history[-3]) < 0.001} \
            {(val_history[-3] - val_history[-4]) < 0.001} {(val_history[-4] - val_history[-5]) < 0.001}')
        
        if len(val_history) > 4 and (val_history[-1] - val_history[-2]) < 0.001  and \
                                    (val_history[-2] - val_history[-3]) < 0.001 and \
                                    (val_history[-3] - val_history[-4]) < 0.001 and \
                                    (val_history[-4] - val_history[-5]) < 0.001:
            print('pruned')
            return loss_history, train_history, val_history, best_model_name
        
    return loss_history, train_history, val_history, best_model_name
        
    
def compute_accuracy(model, loader, device):
    """
    Computes accuracy on the dataset wrapped in a loader    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval()
    correct_samples = 0
    total_samples = 0 
    for i_step, (id, visual_embeds, text, y) in enumerate(loader):
        y = y.to(device)
        visual_embeds = visual_embeds.to(device)    
        tokens = tokenizer(list(text), padding='max_length', max_length=77)

        input_ids = torch.tensor(tokens["input_ids"], device=device)
        attention_mask = torch.tensor(tokens["attention_mask"], device=device)
        token_type_ids = torch.tensor(tokens["token_type_ids"], device=device)

        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)


        outputs = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                          token_type_ids=token_type_ids, 
                          visual_embeds=visual_embeds, 
                          visual_attention_mask=visual_attention_mask, 
                          visual_token_type_ids=visual_token_type_ids
                      )
    
        prediction = outputs.prediction_logits.sum(axis=1)
            
        _, indices = torch.max(prediction, 1)
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]            

    val_accuracy = float(correct_samples) / total_samples
         
    return val_accuracy

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
features_train_dataset = FeaturesDataset(visual_embeddings_train, train_dict)
features_val_dataset = FeaturesDataset(visual_embeddings_val, val_dict)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
loss = torch.nn.CrossEntropyLoss()

In [11]:

if False:
    layer_count=189

    step_size=4
    batch_size=400
    learning_rate=0.0036137
    gamma=0.63426

    model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre') # this checkpoint has 1024 dimensional visual embeddings projection
    for i, param in enumerate(model.parameters()):
        param.requires_grad = False
        if i > layer_count:
            break

    model.cls.predictions.decoder = torch.nn.Linear(in_features=768, out_features=2, bias=True)
    model = model.to(device)


    params = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params.append(param)        

    for i, (name, param) in enumerate(model.named_parameters()):
        if param.requires_grad == True:
            print(i, name)

    optimizer = torch.optim.Adam(params, lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    loss_history, train_history, val_history, best_model_name = train_model(
            model, 
            DataLoader(features_train_dataset, batch_size=batch_size),
            DataLoader(features_val_dataset, batch_size=500),
            loss, optimizer, 2, scheduler, device)


#189 visual_bert.encoder.layer.11.attention.self.value.weight
#190 visual_bert.encoder.layer.11.attention.self.value.bias
#191 visual_bert.encoder.layer.11.attention.output.dense.weight
#192 visual_bert.encoder.layer.11.attention.output.dense.bias
#193 visual_bert.encoder.layer.11.attention.output.LayerNorm.weight
#194 visual_bert.encoder.layer.11.attention.output.LayerNorm.bias
#195 visual_bert.encoder.layer.11.intermediate.dense.weight
#196 visual_bert.encoder.layer.11.intermediate.dense.bias
#197 visual_bert.encoder.layer.11.output.dense.weight
#198 visual_bert.encoder.layer.11.output.dense.bias

In [12]:
#assert False

In [13]:
epoch_count = 30

In [ ]:
def objective(trial):        
    layer_count = trial.suggest_int("layer_count", 189, 211)    
    step_size = trial.suggest_int("step_size", 4, 10, 2)      
    batch_size = trial.suggest_int("batch_size", 32, 2080, 64)    
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2)
    gamma = trial.suggest_float("gamma", 0.5, 1)
    
    model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre') # this checkpoint has 1024 dimensional visual embeddings projection
    for i, param in enumerate(model.parameters()):
        param.requires_grad = False
        if i > layer_count:
            break

    model.cls.predictions.decoder = torch.nn.Linear(in_features=768, out_features=2, bias=True)
    model = model.to(device)


    params = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params.append(param)        

    optimizer = torch.optim.Adam(params, lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    try:
        loss_history, train_history, val_history, best_model_name = train_model(
            model, 
            DataLoader(features_train_dataset, batch_size=batch_size),
            DataLoader(features_val_dataset, batch_size=500),
            loss, optimizer, epoch_count, scheduler, device)
    
    except Exception as ex:
        print('Exception:', ex)
        return 0
    
    return (np.mean(val_history) + np.max(val_history) + val_history[-1] + len(val_history) / epoch_count / 2) / 4


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

[I 2022-11-07 02:27:57,852] A new study created in memory with name: no-name-0fb75aaf-af88-417d-8cb9-0a417fe7eb60
[I 2022-11-07 02:28:02,901] Trial 0 finished with value: 0.0 and parameters: {'layer_count': 190, 'step_size': 8, 'batch_size': 992, 'learning_rate': 0.007271698825106093, 'gamma': 0.7581052965288542}. Best is trial 0 with value: 0.0.


Exception: CUDA out of memory. Tried to allocate 1.39 GiB (GPU 0; 8.00 GiB total capacity; 5.94 GiB already allocated; 116.50 MiB free; 6.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Epoch: 0; 35.74 sec; lr: 0.003929; Average loss: 125.08, Train accuracy: 0.5382, Val accuracy: 0.5000
Epoch: 1; 71.72 sec; lr: 0.003929; Average loss: 32.30, Train accuracy: 0.5132, Val accuracy: 0.5000
Epoch: 2; 108.03 sec; lr: 0.003929; Average loss: 15.60, Train accuracy: 0.5529, Val accuracy: 0.5000
Epoch: 3; 144.55 sec; lr: 0.003929; Average loss: 17.34, Train accuracy: 0.5352, Val accuracy: 0.5000


[I 2022-11-07 02:31:06,540] Trial 1 finished with value: 0.3958333333333333 and parameters: {'layer_count': 189, 'step_size': 10, 'batch_size': 160, 'learning_rate': 0.003929065374073511, 'gamma': 0.6560002701832426}. Best is trial 1 with value: 0.3958333333333333.


Epoch: 4; 181.31 sec; lr: 0.003929; Average loss: 11.64, Train accuracy: 0.5354, Val accuracy: 0.5000
True True             True True
pruned


[I 2022-11-07 02:31:09,875] Trial 2 finished with value: 0.0 and parameters: {'layer_count': 207, 'step_size': 8, 'batch_size': 1376, 'learning_rate': 0.005898138589999307, 'gamma': 0.6635077170713892}. Best is trial 1 with value: 0.3958333333333333.


...

Epoch: 29; 1010.91 sec; lr: 0.000209; Average loss: 6.66, Train accuracy: 0.5995, Val accuracy: 0.5181
False True             False True


[I 2022-11-07 22:15:41,727] Trial 135 finished with value: 0.0 and parameters: {'layer_count': 209, 'step_size': 4, 'batch_size': 672, 'learning_rate': 0.00990208335743509, 'gamma': 0.6121902024243027}. Best is trial 127 with value: 0.5431559571619813.


Exception: CUDA out of memory. Tried to allocate 964.00 MiB (GPU 0; 8.00 GiB total capacity; 4.49 GiB already allocated; 0 bytes free; 6.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Epoch: 0; 33.80 sec; lr: 0.008919; Average loss: 321.08, Train accuracy: 0.5672, Val accuracy: 0.5000
Epoch: 1; 67.54 sec; lr: 0.008919; Average loss: 185.33, Train accuracy: 0.5585, Val accuracy: 0.4980
Epoch: 2; 101.21 sec; lr: 0.008919; Average loss: 159.81, Train accuracy: 0.5483, Val accuracy: 0.5040
Epoch: 3; 134.90 sec; lr: 0.005270; Average loss: 78.14, Train accuracy: 0.5786, Val accuracy: 0.5100
Epoch: 4; 168.66 sec; lr: 0.005270; Average loss: 63.77, Train accuracy: 0.5523, Val accuracy: 0.5221
False False             False True
Epoch: 5; 202.36 sec; lr: 0.005270; Average loss: 32.74, Train accuracy: 0.5910, Val accuracy: 0.5100
True False   

[I 2022-11-07 22:32:33,928] Trial 136 finished with value: 0.5277443105756359 and parameters: {'layer_count': 209, 'step_size': 4, 'batch_size': 544, 'learning_rate': 0.008919415838452821, 'gamma': 0.5908091212665064}. Best is trial 127 with value: 0.5431559571619813.


Epoch: 29; 1010.19 sec; lr: 0.000224; Average loss: 6.81, Train accuracy: 0.6152, Val accuracy: 0.5422
False False             True False
Epoch: 0; 33.65 sec; lr: 0.008755; Average loss: 580.23, Train accuracy: 0.5235, Val accuracy: 0.5000
Epoch: 1; 67.18 sec; lr: 0.008755; Average loss: 205.63, Train accuracy: 0.5833, Val accuracy: 0.5020
Epoch: 2; 100.81 sec; lr: 0.008755; Average loss: 128.78, Train accuracy: 0.5574, Val accuracy: 0.5020
Epoch: 3; 134.37 sec; lr: 0.005190; Average loss: 85.53, Train accuracy: 0.5423, Val accuracy: 0.5301
Epoch: 4; 167.98 sec; lr: 0.005190; Average loss: 25.84, Train accuracy: 0.6095, Val accuracy: 0.5321
False False             True False
Epoch: 5; 201.51 sec; lr: 0.005190; Average loss: 27.00, Train accuracy: 0.5782, Val accuracy: 0.5643
False False             False True
Epoch: 6; 235.12 sec; lr: 0.005190; Average loss: 18.83, Train accuracy: 0.5922, Val accuracy: 0.5301
True False             False False
Epoch: 7; 268.63 sec; lr: 0.003076; Averag

In [ ]:
print("Value:", study.best_value)
print("Parameters:", study.best_params)

In [ ]:
#Value: 0.5200803212851406
#Parameters: {'layer_count': 209, 'step_size': 4, 'batch_size': 544, 'learning_rate': 0.0036136998392606355, 'gamma': 0.6342624350988135}

In [ ]:
assert False

In [ ]:
def objective(trial):    
    if trial.should_prune():
        print("pruned")
        raise optuna.exceptions.TrialPruned()
        
    #layer_count = trial.suggest_int("layer_count", 180, 212, 2)    
    step_size = trial.suggest_int("step_size", 4, 10, 2)      
    batch_size = trial.suggest_int("batch_size", 32, 544, 64)    
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2)
    gamma = trial.suggest_float("gamma", 0.5, 1)
    
    model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre') # this checkpoint has 1024 dimensional visual embeddings projection
    for i, param in enumerate(model.parameters()):
        param.requires_grad = False
        #if i > layer_count:
        #    break

    model.cls.predictions.decoder = torch.nn.Linear(in_features=768, out_features=2, bias=True)
    model = model.to(device)


    params = []
    for name, param in model.named_parameters():
        if param.requires_grad == True:
            params.append(param)        

    optimizer = torch.optim.Adam(params, lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    try:
        loss_history, train_history, val_history, best_model_name = train_model(
            model, 
            DataLoader(features_train_dataset, batch_size=batch_size),
            DataLoader(features_val_dataset, batch_size=500),
            loss, optimizer, 50, scheduler, device)
    
    except Exception as ex:
        print('Exception:', ex)
        return 0
    
    return val_history[-1]


study = optuna.create_study(direction="maximize", pruner=optuna.pruners.PatientPruner(None, patience=4, min_delta=0.01))
study.optimize(objective, n_trials=100)

In [ ]:
#model = VisualBertForPreTraining.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre') # this checkpoint has 1024 dimensional visual embeddings projection

In [ ]:
#for param in model.parameters():
#    param.requires_grad = False

In [ ]:
#model

In [ ]:
#model.cls.predictions.decoder = torch.nn.Linear(in_features=768, out_features=2, bias=True)

In [ ]:
#model = model.to(device)

In [ ]:
#for name, param in model.named_parameters():
#    if param.requires_grad == True:
#        print(name)

In [ ]:
for id, visual_embeds, text, label in DataLoader(features_val_dataset, batch_size=9):

    visual_embeds = visual_embeds.to(device)    
    tokens = tokenizer(list(text), padding='max_length', max_length=77)

    input_ids = torch.tensor(tokens["input_ids"], device=device)
    attention_mask = torch.tensor(tokens["attention_mask"], device=device)
    token_type_ids = torch.tensor(tokens["token_type_ids"], device=device)

    visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
    visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
        
    
    
    
    outputs = model(input_ids=input_ids, 
                      attention_mask=attention_mask, 
                      token_type_ids=token_type_ids, 
                      visual_embeds=visual_embeds, 
                      visual_attention_mask=visual_attention_mask, 
                      visual_token_type_ids=visual_token_type_ids
                  )
    
    prediction = outputs.prediction_logits.sum(axis=1)
    break
    
prediction.shape

In [ ]:
prediction.sum(axis=1).shape

In [ ]:
t1 = time.time()
val_accuracy = compute_accuracy(model, DataLoader(features_val_dataset, batch_size=500), device)
print("%f sec; Val accuracy: %f" % (round(time.time() - t1, 2), val_accuracy))

In [ ]:
assert False

In [ ]:
len(list(model.parameters()))

In [ ]:
loss = torch.nn.CrossEntropyLoss()

params = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params.append(param)
        
optimizer = torch.optim.Adam(params, lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

loss_history, train_history, val_history, best_model_name = train_model(
    model, 
    DataLoader(features_train_dataset, batch_size=500),
    DataLoader(features_val_dataset, batch_size=500),
    loss, optimizer, 50, scheduler, device)
print('end!')


In [ ]:
fig = plt.figure(figsize=(10, 8))    
plt.xlabel("#iteration")
plt.ylabel("loss")
plt.plot(loss_history, label='loss')
plt.plot(train_history, label='train accuracy')
plt.plot(val_history, label='val accuracy')
fig.legend()
plt.show()

In [ ]:
print("best model:", best_model_name)

best_model = torch.load(open(best_model_name, 'rb'))
print(best_model)

In [ ]:
best_model.eval()
for i_step, (x, y) in enumerate(DataLoader(features_val_dataset, batch_size=5000)):
    prediction = best_model(x)

acc_score = accuracy_score(np.array([x.item() for x in labels_val]), torch.max(prediction, 1)[1])
auc_score = roc_auc_score(np.array([x.item() for x in labels_val]), prediction[:,1].detach().numpy())

fpr, tpr, thresh = roc_curve(labels_val, prediction[:,1].detach().numpy(), pos_label=1)

random_probs = [0 for i in range(len(labels_val))]
p_fpr, p_tpr, _ = roc_curve(labels_val, random_probs, pos_label=1)
auc_score = roc_auc_score(labels_val, prediction[:,1].detach().numpy())

print('Accuracy: ', acc_score, '\n', 'ROC AUC: ', auc_score, sep='')

plt.plot(fpr, tpr, linestyle='--',color='orange')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
plt.title('ROC Curve', fontsize=20)
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive rate',fontsize=18)

plt.show();

In [ ]:
assert False

In [ ]:
outputs_val = {}

for i, k in enumerate(tqdm(visual_embeddings_val)):

    outputs = model(input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    token_type_ids=token_type_ids, 
                    visual_embeds=visual_embeds, 
                    visual_attention_mask=visual_attention_mask, 
                    visual_token_type_ids=visual_token_type_ids)
    
    o = torch.tensor(outputs.prediction_logits.cpu().flatten(), dtype=torch.float16)
    outputs_val[k] = o
    
    if (i + 1) % 50 == 0:        
        with open(f'd:\\val\\outputs_val_{i}.pkl', 'wb') as f:
            pickle.dump(outputs_val, f)
        outputs_val = {}

if len(outputs_val) > 0:
    with open(f'd:\\val\\outputs_val_{i}.pkl', 'wb') as f:
        pickle.dump(outputs_val, f)

In [ ]:
#assert False

In [ ]:
with open('d:\\visual_embeddings_train.pkl', 'rb') as f:
    visual_embeddings_train = pickle.load(f)
    
train_dict = {}
for x in train_data.values:
    if x[1] in visual_embeddings_train:
        train_dict[x[1]] = {'label':x[2], 'text':x[3]}
    
with open('d:\\train\\labels_train.pkl', 'wb') as f:
    pickle.dump(train_dict, f)

In [ ]:
outputs_train = {}

for i, k in enumerate(tqdm(visual_embeddings_train)):
    visual_embeds = visual_embeddings_train[k][0].to(device)
    text = train_dict[k]['text']
    
    tokens = tokenizer([text], padding='max_length', max_length=77)
    
    input_ids = torch.tensor(tokens["input_ids"], device=device)
    attention_mask = torch.tensor(tokens["attention_mask"], device=device)
    token_type_ids = torch.tensor(tokens["token_type_ids"], device=device)
    
    visual_embeds = torch.stack([visual_embeds])
    visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
    visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long, device=device)
    
    outputs = model(input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    token_type_ids=token_type_ids, 
                    visual_embeds=visual_embeds, 
                    visual_attention_mask=visual_attention_mask, 
                    visual_token_type_ids=visual_token_type_ids)
    
    o = torch.tensor(outputs.prediction_logits.cpu().flatten(), dtype=torch.float16)
    outputs_train[k] = o
    
    if (i + 1) % 50 == 0:        
        with open(f'd:\\train\\outputs_train_{i}.pkl', 'wb') as f:
            pickle.dump(outputs_train, f)
        outputs_train = {}
        
if len(outputs_train) > 0:
    with open(f'd:\\train\\outputs_train_{i}.pkl', 'wb') as f:
        pickle.dump(outputs_train, f)